In [1]:
import pandas as pd
import numpy as np
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import(
    classification_report, confusion_matrix, accuracy_score, mean_squared_error, mean_absolute_error
)
from sklearn.preprocessing import (
    StandardScaler, LabelBinarizer, FunctionTransformer,PolynomialFeatures, OrdinalEncoder
)
# from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.pipeline import Pipeline, FeatureUnion

import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor,RegressorChain
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import RegressorChain
from sklearn_pandas import DataFrameMapper

In [18]:
# from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVR, SVR


In [3]:
aa = pd.read_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\AA-NACL.csv',header=0,
                    names=['temp','conductivity','velocity','nacl','aa'],dtype='float32')
df = pd.read_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\AA_NACL_NewFeatures_Corr.csv',dtype='float64')
df_test = pd.read_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\AA_NACL_NewFeatures_Corr_test.csv',dtype='float64')


In [4]:
aa_shuffled = aa.sample(frac=1)

In [59]:
X_train = aa_shuffled.loc[:,['temp','conductivity','nacl']]
X_train

,temp,conductivity,nacl
72,15.0,14.480000,1.0
67,15.0,1.080000,0.0
69,15.0,1.330000,0.0
85,25.0,0.360000,0.0
17,0.0,25.830000,3.0
...,...,...,...
52,10.0,9.420000,1.0
66,15.0,0.680000,0.0
60,10.0,25.360001,3.0
94,25.0,17.850000,1.0


In [60]:
y_train = aa_shuffled.loc[:,['aa']]
y_train


,aa
72,1.0
67,2.0
69,3.0
85,0.5
17,1.5
...,...
52,1.5
66,1.5
60,2.0
94,1.5


In [61]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,test_size=0.2,random_state=42)

In [62]:
X_train_org = X_train.copy()
X_test_org = X_test.copy()
y_train_org = y_train.copy()
y_test_org = y_test.copy()

In [73]:
X_train_org.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\train-test dataset\\X_train.csv')
X_test_org.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\train-test dataset\\X_test.csv')
y_train_org.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\train-test dataset\\y_train.csv')
y_test_org.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\train-test dataset\\y_test.csv')

In [63]:
class TitanicTransformer(TransformerMixin):
    def transform(self, X, **transform_params):
        X['con-na'] = X['conductivity']/np.mean(X['nacl'])
        return X
    def fit(self, X, y=None, **fit_params):
        return self

In [72]:
mapper = DataFrameMapper([
#     (['temp'],  [SimpleImputer(strategy='mean'), StandardScaler()]),
    (['temp'],  [SimpleImputer(strategy='mean'), OrdinalEncoder()]),
    (['conductivity'],  [SimpleImputer(strategy='mean'), StandardScaler()]),
    (['nacl'],  [SimpleImputer(strategy='mean'), StandardScaler()])
], df_out=True)

In [65]:
pipe1a = Pipeline([
    ('map', mapper)
#     ('feature_gen', TitanicTransformer())
])

In [66]:
pipe1a = pipe1a.fit(X_train)
X_train = pipe1a.transform(X_train)

In [67]:
X_test =  pipe1a.transform(X_test)

In [68]:
# y_test.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\aa-results\\y_train.csv')

In [69]:
def run_regression(X_train, y_train,X_test,y_test):
    ESTIMATORS = {
        "Linear regression": LinearRegression(),
        "RandomForestRegressor": RandomForestRegressor(max_depth=4, random_state=2),
        "Decision Tree Regressor":DecisionTreeRegressor(max_depth=5),
        "SVR":SVR(),
        "LinearSVR":LinearSVR(),
#         "RegressorChain Regressor":RegressorChain(DecisionTreeRegressor()),
#         "MultiOutputRegressor Regressor":MultiOutputRegressor(LinearRegression()),
#         "RegressorChain_Linear Regressor":RegressorChain(LinearRegression()),
#         "MultiOutput_DecisionTree Regressor":MultiOutputRegressor(DecisionTreeRegressor())
    }
    # 9.1 Create an empty dictionary to collect prediction values
    y_test_predict = dict()
    y_mse = dict()
    for name, estimator in ESTIMATORS.items():     
        estimator.fit(X_train.values, y_train.values.ravel())                    # fit() with instantiated object
        y_test_predict[name] = estimator.predict(X_test.values)   # Make predictions and save it in dict under key: name
        y_mse[name] = mean_squared_error(y_test.values, estimator.predict(X_test.values))
        print('Predicted: %s' % y_mse[name])
    #     print('Predicted: %s' %  y_test_predict[name])
    return y_test_predict
#     return y_test_predict.items()
    
    #     scoring = ['precision_macro', 'recall_macro']
#     cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# #     n_scores = cross_validate(wrapper, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
#     n_scores = cross_val_score(wrapper, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
#     n_scores = np.absolute(n_scores)
#     # summarize performance
#     print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [70]:
X_train

,temp,conductivity,nacl
29,1.0,-0.369854,1.0
4,0.0,-1.041640,0.0
97,4.0,0.080177,1.0
44,2.0,-0.925545,0.0
5,0.0,-1.028596,0.0
...,...,...,...
10,0.0,-0.463122,1.0
13,0.0,-0.470296,1.0
12,0.0,-0.468992,1.0
86,4.0,-1.036422,0.0


In [71]:
y_test_predict = run_regression(X_train, y_train,X_test,y_test)

Predicted: 1.2118107092954171
Predicted: 1.1210875335166262
Predicted: 1.3233206372669934
Predicted: 1.3416863770619787
Predicted: 1.1790057363749333


C:\Program Data\envs\Tensorflow\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [66]:
y_test

,aa
2,1.0
26,2.8
79,1.0
77,0.2
45,1.5
96,2.8
12,2.8
32,2.0
9,1.0
31,1.5


In [90]:
y_test_predict

{'Linear regression': array([1.5911808, 1.5644236, 1.4541442, 1.458376 , 1.5143303, 1.4308635,
        1.5523394, 1.5320908, 1.5556402, 1.5324293, 1.5614613, 1.4619443,
        1.446264 , 1.3787376, 1.4015809, 1.4491507, 1.3770449, 1.4579663,
        1.474634 , 1.4793692, 1.42934  ], dtype=float32),
 'RandomForestRegressor': array([0.61402381, 2.65796718, 1.57309558, 1.53941684, 1.29176665,
        1.29357792, 1.46815145, 1.53401524, 1.31835721, 1.53401524,
        1.47959707, 1.35902752, 1.54605381, 1.4578784 , 2.53506514,
        1.60537891, 1.4919284 , 1.62652185, 1.41817727, 1.29251609,
        1.34648723]),
 'Decision Tree Regressor': array([0.5       , 2.63999999, 1.54166666, 1.54166666, 1.5       ,
        1.4       , 1.4       , 1.4       , 1.4       , 1.4       ,
        1.5       , 1.4       , 1.54166666, 1.4       , 2.63999999,
        1.54166666, 1.4       , 1.4       , 1.4       , 1.4       ,
        1.4       ]),
 'SVR': array([1.39963172, 1.27351404, 1.59999818, 1.600170